In [2]:
import numpy as np    
import pandas as pd    
import matplotlib.pyplot as plt  
import os  
import cartopy.crs as ccrs  
import cartopy.feature as cfeature  
import matplotlib.ticker as mticker  
from matplotlib.ticker import MultipleLocator, FuncFormatter

In [3]:
df_2024 = pd.read_excel(r'D:\ze_1112\最新数据\NORC2022-09_Chl-a+营养盐_20240422更新_绘图使用数据.xls')
# df.dropna(inplace=True)
df_2024 = df_2024.rename(columns={'站位':'站位名'})
df_2021 = pd.read_excel(r'E:\all_draw\CTD\营养盐数据.xlsx')
df_2021['站位'] = df_2021['站位'].fillna(method='ffill')
df_2021 = df_2021.rename(columns={'站位':'站位名'})


C:\Users\Administrator\AppData\Local\Temp\ipykernel_21608\2144448703.py:5: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_2021['站位'] = df_2021['站位'].fillna(method='ffill')


In [4]:
stations_2021 = []
for i in df_2021['站位名']:
    stations_2021.append(i)
stations_2021 = list(set(stations_2021))
print(stations_2021)
stations_2024 = []
for i in df_2024['站位名']:
    stations_2024.append(i)
stations_2024 = list(set(stations_2024))
print(stations_2024)

['N3-01', 'E163-08', 'QB01', nan, 'FB01', 'QB17', 'N3-05', 'E163-07', 'QB16', 'E142-10', 'E142-06', 'EQ-03', 'E163-09', 'QB15', 'E163-11', 'QB04', 'QB12', 'E163-06', 'E163-05', 'E163-02', 'QB14', 'QB10', 'E142-07', 'QB-07new', 'E163-04', 'E142-5', 'E163-03', 'QB03', 'QB06', 'QB05', 'QB09', 'QB11', 'E163-10', 'QB07', 'QB19', 'E163-01', 'QB02', 'FB02', 'N3-03', 'Mk-03', 'QB18', 'QB08']
['E130-12', 'FB01', 'E163-06s', 'N21-10', 'E163-07', 'E130-07', 'E130-04', 'E130-03', 'E130-02', 'QB04', 'E163-06', 'N21-12', 'E163-05', 'E140-07', 'N03-04', 'N03-02', 'E163-02', 'E130-09', 'E163-04s', 'E163-04', 'E130-14', 'E163-03', 'QB03', 'QB06', 'E140-08', 'N21-13', 'QB07', 'E163-01', 'E140-03orQB01', 'N21-11', 'E130-08', 'E130-05', 'E130-11', 'E130-10', 'E130-13', 'N21-14', 'QB08', 'N03-03', 'E130-06', 'E140-11']


In [5]:
import re

def dms_to_decimal(dms):
    # 使用正则表达式匹配度分秒格式，包括不同的分隔符和方向指示符
    match = re.match(r"(\d+)[°\s](\d+)[′'’\s](\d+(\.\d+)?)[\"\s]?(N|S|E|W)?", dms)
    if match:
        degrees = float(match.group(1))
        minutes = float(match.group(2))
        seconds = float(match.group(3))
        direction = match.group(5)
        decimal = degrees + (minutes / 60) + (seconds / 3600)
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        raise ValueError(f"无法解析的度分秒格式: {dms}")

# 自定义刻度格式化函数
def lon_formatter(x, pos):
    if x < 0:
        return f'{abs(int(x))}°W'
    elif x > 180:
        return f'{360-int(x)}°W'
    else:
        return f'{int(x)}°E'

def lat_formatter(x, pos):
    if x > 0:
        return f'{int(x)}°N' 
    else:
        return f'{int(x)}°S'

def dms_to_decimal_2024(dms):
    """将度分秒转换为十进制度数"""
    dms = dms.replace('°', ' ').replace('′', ' ').replace('″', ' ').replace('"E', ' ').replace('"N', ' ').replace('"S', ' ')
    parts = dms.split()
    degrees = float(parts[0])
    minutes = float(parts[1]) if len(parts) > 1 else 0
    seconds = float(parts[2]) if len(parts) > 2 else 0
    decimal = degrees + (minutes / 60) + (seconds / 3600)
    return decimal

In [6]:
stations_ctd_2021 = os.listdir(r'E:\all_draw\CTD\CTD')
lonlat = []  
for i in stations_ctd_2021:
    if i.endswith('.xls'):
        df = pd.read_excel(r'E:\all_draw\CTD\CTD\%s' % i, skiprows=1, header=1)
        df = df.drop(0)
        longitude = df['经度'].apply(dms_to_decimal).values # type: ignore
        latitude = df['纬度'].apply(dms_to_decimal).values  # type: ignore
        name = df['站位名'].values
        lonlat.append((longitude[3], latitude[3], name[3]))
lonlat = pd.DataFrame(lonlat, columns=['lon', 'lat', '站位名'])
data_path_2021 = pd.merge( lonlat, df_2021, on='站位名')

In [7]:
stations_ctd_2024 = os.listdir(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表')
lonlat = []  
for i in stations_ctd_2024:
    if i.endswith('.xls'):
        df = pd.read_excel(r'D:\ze_1112\最新数据\CTD补上溶解氧_04共享航次计划调查资料汇交表\%s' % i, skiprows=1, header=1)
        df = df.drop(0)
        longitude = df['经度'].apply(dms_to_decimal_2024).values # type: ignore
        latitude = df['纬度'].apply(dms_to_decimal_2024).values  # type: ignore
        name = df['站位名'].values
        lonlat.append((longitude[3], latitude[3], name[3]))
lonlat = pd.DataFrame(lonlat, columns=['lon', 'lat', '站位名'])
data_path_2024 = pd.merge( lonlat, df_2024, on='站位名')

In [13]:

#####################################绘制scatter###############################################
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
plt.rcParams['font.size'] = 24

stations_path_2021 = []
for i in data_path_2021['站位名']:
    stations_path_2021.append(i)
stations_path_2021 = list(set(stations_path_2021))

stations_path_2024 = []
for i in data_path_2024['站位名']:
    stations_path_2024.append(i)
stations_path_2024 = list(set(stations_path_2024))

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection=ccrs.PlateCarree())
ax.set_extent([135, 165.001, -5, 20.001], crs=ccrs.PlateCarree())

for station_path_2021 in stations_path_2021:
    data = data_path_2021[data_path_2021['站位名'] == station_path_2021]
    max_depth = data['深度(m)'].max()
    # data_2021 = data_path_2021[(data_path_2021['深度(m)'] >= 4800) & (data_path_2021['深度(m)'] <= 5200)]
    data_2021 = data_path_2021[data_path_2021['深度(m)'] == 500]##########################################################################################
    station = data_2021['站位名']
    lon = data_2021['lon'].values
    lat = data_2021['lat'].values
    sio4_2021 = data_2021['硅酸盐'].values
    sc = ax.scatter(lon, lat, s=100, c=sio4_2021, cmap='RdYlBu_r', marker='o', alpha=1, edgecolors='k', transform=ccrs.PlateCarree(), zorder=10, linewidths=0.5, vmin=24, vmax=55)

for station_path_2024 in stations_path_2024:
    data = data_path_2024[data_path_2024['站位名'] == station_path_2024]
    max_depth = data['水深（m）'].max()
    # data_2024 = data_path_2024[(data_path_2024['水深（m）'] >= 4800) & (data_path_2024['水深（m）'] <= 5200)]
    data_2024 = data_path_2024[data_path_2024['水深（m）'] == 500]##########################################################################################
    station = data_2024['站位名']
    long = data_2024['lon'].values
    lati = data_2024['lat'].values
    sio4_2024 = data_2024['SiO4-Si  (μmol/L)'].values
    sc2 = ax.scatter(long+0.3, lati+0.3, s=100, c=sio4_2024, cmap='RdYlBu_r', marker='^', alpha=1, edgecolors='gray', transform=ccrs.PlateCarree(), zorder=10, linewidths=0.5, vmin=24, vmax=55)
    
ax.spines[:].set_linewidth(2)
ax.add_feature(cfeature.BORDERS, linewidth=2)
ax.add_feature(cfeature.COASTLINE, linewidth=2)
ax.add_feature(cfeature.RIVERS, linewidth=2)
ax.add_feature(cfeature.LAND, edgecolor='black', zorder=9)
#####################################################
import xarray as xr
ds = xr.open_dataset(r"E:\all_draw\data\Copernicus_bathy\cmems_mod_glo_phy_my_0.083deg_static_1720691864705.nc")
elevation = ds['deptho'].values
longitude = ds['longitude'].values
latitude = ds['latitude'].values
mask = elevation > 4000 ##################################################################################################
mask_1 = elevation > 2000
mask_2 = elevation > 3000
zhe_gai = ax.contourf(longitude, latitude, mask, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha = 0.3)
zhe_gai_1 = ax.contourf(longitude, latitude, mask_1, levels=[0, 0.5], colors='orange', transform=ccrs.PlateCarree(), alpha = 0.7)
zhe_gai_2 = ax.contourf(longitude, latitude, mask_2, levels=[0, 0.5], colors='grey', transform=ccrs.PlateCarree(), alpha = 0.7)
######################################################
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
gl = ax.gridlines(draw_labels=False, linewidth=1, color='white', alpha=0.15, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': 15, 'color': 'black'}
gl.ylabel_style = {'size': 15, 'color': 'black'}
ax.set_xticks(np.arange(125, 165.001, 10))
ax.set_yticks(np.arange(0, 25.001, 5))
ax.xaxis.set_major_formatter(FuncFormatter(lon_formatter))
ax.yaxis.set_major_formatter(FuncFormatter(lat_formatter))  
    
cbar = plt.colorbar(sc, ax=ax, orientation='vertical', pad=0.02, aspect=30, shrink=0.65)
cbar.set_label('SiO4 (μmol/L)', fontsize=18)
cbar.set_ticks(np.arange(25, 56, 5))
cbar.set_ticklabels(['25', '30', '35', '40', '45', '50', '55'])
# 添加图例
from matplotlib.lines import Line2D
legend_elements = [Line2D([0], [0], marker='o', color='w', label='2021年数据',
                          markerfacecolor='none', markeredgecolor='k', markersize=10),
                   Line2D([0], [0], marker='^', color='w', label='2024年数据',
                          markerfacecolor='none', markeredgecolor='k', markersize=10)]
ax.legend(handles=legend_elements, loc='upper right', fontsize=12,framealpha=0.8, fancybox=True)


# plt.show()
plt.savefig('2021vs2024_500.png', bbox_inches='tight', dpi=300)
plt.close()

In [9]:
import matplotlib
help(matplotlib.markers)

Help on module matplotlib.markers in matplotlib:

NAME
    matplotlib.markers

DESCRIPTION
    Functions to handle markers; used by the marker functionality of
    `~matplotlib.axes.Axes.plot`, `~matplotlib.axes.Axes.scatter`, and
    `~matplotlib.axes.Axes.errorbar`.

    All possible markers are defined here:

    ============================== ====== =========================================
    marker                         symbol description
    ============================== ====== =========================================
    ``"."``                        |m00|  point
    ``","``                        |m01|  pixel
    ``"o"``                        |m02|  circle
    ``"v"``                        |m03|  triangle_down
    ``"^"``                        |m04|  triangle_up
    ``"<"``                        |m05|  triangle_left
    ``">"``                        |m06|  triangle_right
    ``"1"``                        |m07|  tri_down
    ``"2"``                        |m08|  tri